# Mapping the ocean floor

Here's hoping that my games programming past comes back to help me here.

First of all, we're going to want to examine the floor.  The critical bit of information we need is that for any given pair of coordinates, (x,y) we should be able to ask it for the heights of its neighbours, which will return a list like `[1,1,2,3]` or `[1,1]`.  If we ask an edge or a corner for its neighbours, we'll just return the 2 or 3 neighbours.

Once we have that, we can then iterate over all the neighbours for each cell and ask if they are all greater than the current cell, if they are, we've found a "local minima".

In [1]:
## Import ipytest and get it setup for use in Python Notebook
import pytest
import ipytest
ipytest.autoconfig()

In [15]:
class Grid:
    def __init__(self,width,height,cells):
        self.width = width
        self.height = height
        self.cells = cells
        
    def find_neighbours(self, x,y):
        neighbours = []
        if y > 0:
            neighbours.append(self.cells[(x,y-1)])
        if x < self.width-1:
            neighbours.append(self.cells[(x+1,y)])
        if y < self.height-1:
            neighbours.append(self.cells[(x,y+1)])
        if x > 0:
            neighbours.append(self.cells[(x-1,y)])
        return neighbours

def grid_from_lines(lines):
    height = len(lines)
    width = len(lines[0])
    cells = {}
    for y,line in enumerate(lines):
        for x,cell in enumerate(line):
            cells[(x,y)] = int(cell)
    return Grid(width,height,cells)

test_data = """2199943210
3987894921
9856789892
8767896789
9899965678""".split("\n")

test_grid = grid_from_lines(test_data)
assert 10 == test_grid.width
assert 5 == test_grid.height
assert 2 == test_grid.cells[(0,0)]
assert 1 == test_grid.cells[(1,0)]
assert 3 == test_grid.cells[(0,1)]

# Implement find_neighbours on Grid class here

assert [1,1] == test_grid.find_neighbours(9,0)
assert [9,9,2] == test_grid.find_neighbours(1,0)
assert [8,6,6,8] == test_grid.find_neighbours(2,2)

Right, now we've got a basic grid class, we can find the lowest point.

Sadly, due to the way the Jupyter notebook works, we cannot reopen an existing class, so we have to redefine it if we want to add a method, which we do in this case.  Note that we're interested in the heights of the lowest points.

We could do this two ways, the longer and more readable way is to make find_lowest_points return a list of coordinates, and then write something that maps those into heights and then adds one to each one to get the risk.  Or we can put that into the Grid class as a find_risk_score.  I'm going to do the latter for speed, but I think we might need to come back and create a find_lowest_points later.

In [18]:
class Grid:
    def __init__(self,width,height,cells):
        self.width = width
        self.height = height
        self.cells = cells
        
    def find_neighbours(self, x,y):
        neighbours = []
        if y > 0:
            neighbours.append(self.cells[(x,y-1)])
        if x < self.width-1:
            neighbours.append(self.cells[(x+1,y)])
        if y < self.height-1:
            neighbours.append(self.cells[(x,y+1)])
        if x > 0:
            neighbours.append(self.cells[(x-1,y)])
        return neighbours
    
    def find_risk_score(self):
        score = 0
        for y in range(self.height):
            for x in range(self.width):
                if self.cells[(x,y)] < min(self.find_neighbours(x,y)):
                    score += self.cells[(x,y)]+1
        return score

def grid_from_lines(lines):
    height = len(lines)
    width = len(lines[0])
    cells = {}
    for y,line in enumerate(lines):
        for x,cell in enumerate(line):
            cells[(x,y)] = int(cell)
    return Grid(width,height,cells)

test_grid = grid_from_lines(test_data)

assert 15 == test_grid.find_risk_score()

Great!  Let's try that on prod data

In [ ]:
grid = grid_from_lines([line.strip() for line in open('day9.txt')])